## Sharing Code Libraries Between Jupyter Notebooks

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import parkit as pk

In [ ]:
# For this example, assume this code is executed in notebook 1.
#
# Create task. Add a metadata tag for the task's 'library'. We'll make the
# task run synchronously by default.
@pk.task(
    name = 'mysum',
    metadata = dict(library = 'math'),
    default_sync = True
)
def __sum__(input: pk.Array):
    return sum(input)

In [ ]:
input = pk.Array()
input.extend([1, 2, 3, 4, 5])
pk.bind_task('mysum')(input)

In [ ]:
# This code runs in notebook 2.
#
# Define a function that imports our pre-defined tasks and binds them
# to symbols in the notebook's module.

def import_library(name: str):
    print('importing library', name)
    for task in pk.Namespace('task'):
        if 'library' in task.metadata and task.metadata['library'] == name:
            if pk.bind_symbol(task.namespace, task.name, overwrite = True):
                print('bound symbol:', task.name)

In [ ]:
import_library('math')

In [ ]:
input = pk.Array()
input.extend([1, 2, 3, 4, 5])
mysum(input)

In [ ]:
# Go back to notebook 1.
#
# Let's change the default mode to run asynchronously.
@pk.task(
    name = 'mysum',
    metadata = dict(library = 'math'),
    default_sync = False
)
def __sum__(input: pk.Array):
    return sum(input)

In [ ]:
# Back to notebook 2.
input = pk.Array()
input.extend([1, 2, 3, 4, 5])
trace = mysum(input)
pk.wait_until(lambda: trace.done)
print(trace)

In [ ]:
trace.result

## Zero-copy Access To Numpy Array in Shared Memory

In [ ]:
# We'll extend Dict to support serialization of Numpy arrays without
# pickling.
import numpy as np

class NumpyDict(pk.Dict):
    
    # This is called during the encoding phase and allows us to 
    # record metadata that we can use when decoding an object.
    def get_metadata(self, value):
        if isinstance(value, np.ndarray):
            return dict(
                shape = value.shape,
                dtype = value.dtype
            )
        return None
    
    # If we have metadata we know its a Numpy array. Return a zerop-copy version
    # if we are in an explicit transaction and we are passed a memoryview. Otherwise
    # create an array with a copy of the data.
    def decode_value(self, bytestring, metadata):
        if metadata is None:
            return super().decode_value(bytestring)
        if self.is_transaction and isinstance(bytestring, memoryview):
            return np.ndarray(metadata['shape'], metadata['dtype'], buffer = bytestring)
        return np.ndarray(metadata['shape'], metadata['dtype'], buffer = bytearray(bytestring))

    # If we have a numpy array, send back the bytes data, otherwise call the standard
    # pickle encoder
    def encode_value(self, value):
        if isinstance(value, np.ndarray):
            return value.data
        return super().encode_value(value)

In [ ]:
d = NumpyDict()

In [ ]:
data = np.zeros((10, 10))

In [ ]:
# We now have a dictionary with special (non-pickled) encoding for Numpy arrays.
d['numpy'] = data
d['other'] = [1, 2, 3]

In [ ]:
# Standard pickle encoding
d['other']

In [ ]:
# The array is a zero-copy version of the data stored in shared memory.
with pk.snapshot(d.namespace):
    array = d['numpy']
    print(array)

In [ ]:
# This code will throw an exception since the array is a read-only zero-copy
# version.
with pk.snapshot(d.namespace):
    array = d['numpy']
    array[0] = 1

In [ ]:
# We can get writable copy of the array by setting the zero_copy parameter to
# False.
with pk.snapshot(d.namespace, zero_copy = False):
    array = d['numpy']
    array[0] = 1

In [ ]:
# Retrieving the data in an implicit transaction will always return a writable
# array with a copy of the data.
array = d['numpy']
array[0] = 1